<a href="https://colab.research.google.com/github/Hyorim-Kim/numpi/blob/main/tfc27text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# RNN을 이용한 텍스트 생성
# 문맥을 반영해서 다음 단어를 예측하여 텍스트를 생성 - 다항분류
# 수정하기 ***
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences, to_categorical
import numpy as np
from keras.layers import Embedding, Dense, LSTM, Flatten
from keras.models import Sequential

# text = """거리에 하얀 눈이 쌓이고 있다
# 그의 눈이 발게 빛난다
# 눈이 내리고 있는 오후 거리는 눈이 부시다"""

text = """
은행권이 연말 희망퇴직 실시를 앞두고 희망퇴직 규모 등에 고심하고 있다. ‘이자 장사’로 손쉽게 번 돈으로 성과급·퇴직금 잔치를 벌인다는 비판 여론이 높아진 상황을 의식하지 않을 수 없어서다. 은행권 노조에선 “은행들 모두 눈치만 보는 상황”이라는 말이 나온다.
4일 금융권에 따르면 KB국민은행과 우리은행이 이달 말 희망퇴직을 실시할 전망이다. 현재 임단협 협상을 진행 중이다. 앞서 NH농협은행은 지난달 21~23일까지 희망퇴직 신청을 받았다.
매년 정례화된 은행권의 희망퇴직이 올해 유독 주목받는 것은 은행들이 사회적 책임을 소홀히 하면서 성과급은 물론 퇴직금 잔치까지 벌인다고 정부와 여론으로부터 뭇매를 맞고 있기 때문이다. 실제로 지난해 5대 은행에서만 2357명의 희망 퇴직자가 1인당 평균 3억5547만원의 특별 퇴직금을 손에 쥐고 은행을 떠났다. 올해 상반기 희망 퇴직자들 중에는 퇴직금 총액이 11억원을 넘는 경우도 있었다.
은행들은 경영·인력 구조 효율화를 이유로 대대적으로 희망퇴직을 실시하고, 은행원들은 희망퇴직 조건이 좋을 때 그만두고 새로운 도전을 하려는 계산이 맞물리면서 희망퇴직자는 계속 늘어나는 추세다. 신한은행은 연초에 이어 지난 8월 두 번째 희망퇴직을 실시하면서 대상을 ‘근속연수 15년 이상, 1983년생 이전 출생자’까지 넓히기도 했다. 만 나이로 30대가 희망퇴직 대상자에 포함됐던 것이다.
은행권도 이런 ‘돈 잔치’ 비판을 의식해 눈치를 보며 고심 중이다. 실적이 좋은데 희망퇴직 조건을 무턱대고 축소했다가 노조가 반발할 수 있고, 그렇다고 최근 분위기를 고려했을 때 신경쓰지 않는 듯한 모습을 보이기도 어렵기 때문이다. KB국민·신한·하나·우리·NH농협은행 등 5대 은행은 올 들어 3분기까지 30조원이 넘는 이자 이익(30조9366억원)을 거뒀다. 작년 같은 기간보다 7.4% 늘어난 것이다.
한 시중은행 노조 관계자는 “직원들이 고생해서 만들어 놓은 성과가 수치(실적)로 확인되는 만큼 직원에게 보상을 해주는 데 대해 이견은 없다”면서도 “하지만 외부 시선도 감안해야 하니 어떻게 풀어나갈 지는 열심히 논의를 하는 상황”이라고 말했다. 다른 노조 관계자도 “아직 희망퇴직 조건이 구체적으로 나온 것은 없다”며 “모두 눈치만 보는 상황”이라고 전했다.
지난달 희망퇴직 신청을 받은 NH농협은행의 경우 희망퇴직 조건을 다소 축소했다. 지난해는 만 56세 이상 직원은 28개월 치, 10년 이상 근속한 만 40세 이상 일반 직원에겐 20~39개월 치 특별 퇴직금을 지급하는 조건이었으나, 올해는 일반 직원은 차등 없이 최대 20개월치 위로금을 준다.
"""

tok = Tokenizer()
tok.fit_on_texts([text])
encoded = tok.texts_to_sequences([text])[0]
print(encoded)
print(tok.word_index)

vocab_size = len(tok.word_index) + 1


[29, 30, 1, 31, 32, 1, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 5, 51, 52, 53, 54, 55, 6, 7, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 3, 66, 67, 68, 69, 70, 71, 8, 72, 73, 9, 74, 75, 1, 10, 76, 77, 78, 79, 80, 11, 81, 82, 12, 83, 84, 85, 86, 87, 88, 89, 13, 90, 91, 92, 93, 94, 95, 96, 14, 97, 98, 15, 99, 100, 16, 101, 102, 103, 104, 17, 18, 105, 106, 107, 108, 11, 109, 16, 110, 111, 13, 112, 113, 19, 114, 115, 116, 117, 118, 119, 120, 121, 3, 122, 123, 1, 20, 124, 21, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 3, 142, 143, 144, 145, 2, 146, 147, 148, 149, 150, 4, 151, 152, 1, 153, 154, 22, 155, 156, 157, 158, 159, 160, 161, 162, 163, 8, 164, 165, 1, 23, 166, 167, 168, 169, 5, 170, 171, 172, 173, 174, 21, 175, 176, 177, 178, 179, 180, 14, 181, 182, 15, 183, 184, 185, 186, 187, 19, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 22, 199, 200, 24, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213,

In [6]:
# 훈련 데이터
sequences = list()
for line in text.split('\n'):   # 문장 토큰화
  enco = tok.texts_to_sequences([line])[0]
  # print(enco) # [2, 3, 1, 4, 5][6, 1, 7, 8][1, 9, 10, 11, 12, 1, 13]
  for i in range(1, len(enco)):
    sequ = enco[:i + 1]
    # print(sequ)
    sequences.append(sequ)

print('학습에 참여할 샘플 수 : %d'%len(sequences))
print(sequences)
max_len = max(len(i) for i in sequences)  # 7

psequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(psequences)

x = psequences[:, :-1]
y = psequences[:, -1]
print(x)
print(y)
y = to_categorical(y, num_classes=vocab_size)
print(y[:2])

학습에 참여할 샘플 수 : 293
[[29, 30], [29, 30, 1], [29, 30, 1, 31], [29, 30, 1, 31, 32], [29, 30, 1, 31, 32, 1], [29, 30, 1, 31, 32, 1, 33], [29, 30, 1, 31, 32, 1, 33, 34], [29, 30, 1, 31, 32, 1, 33, 34, 35], [29, 30, 1, 31, 32, 1, 33, 34, 35, 36], [29, 30, 1, 31, 32, 1, 33, 34, 35, 36, 37], [29, 30, 1, 31, 32, 1, 33, 34, 35, 36, 37, 38], [29, 30, 1, 31, 32, 1, 33, 34, 35, 36, 37, 38, 39], [29, 30, 1, 31, 32, 1, 33, 34, 35, 36, 37, 38, 39, 40], [29, 30, 1, 31, 32, 1, 33, 34, 35, 36, 37, 38, 39, 40, 41], [29, 30, 1, 31, 32, 1, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42], [29, 30, 1, 31, 32, 1, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43], [29, 30, 1, 31, 32, 1, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44], [29, 30, 1, 31, 32, 1, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45], [29, 30, 1, 31, 32, 1, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46], [29, 30, 1, 31, 32, 1, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47], [29, 30, 1, 31, 32, 1, 33, 34, 35, 36, 37, 38, 39, 40, 

In [7]:
model= Sequential()
model.add(Embedding(vocab_size, 32, input_length=max_len-1))
model.add(LSTM(32,activation='tanh'))
model.add(Flatten())
model.add(Dense(32,activation='tanh'))
model.add(Dense(32,activation='relu'))
model.add(Dense(vocab_size,activation='softmax'))
print(model.summary())
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x,y, epochs=200, verbose=2)
print('model.evaluate :', model.evaluate(x,y))


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 52, 32)            8384      
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 flatten_1 (Flatten)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 32)                1056      
                                                                 
 dense_4 (Dense)             (None, 32)                1056      
                                                                 
 dense_5 (Dense)             (None, 262)               8646      
                                                                 
Total params: 27462 (107.27 KB)
Trainable params: 2746

In [8]:
# 문자열 생성
def sequence_gen_func(model, t, current_word, n):
  init_word = current_word
  sentence= ''
  for _ in range(n):
    encoded = t.texts_to_sequences([current_word])[0]
    encoded = pad_sequences([encoded], maxlen=max_len-1,padding='pre')
    result = np.argmax(model.predict(encoded, verbose=0), axis=-1)
    # 예측 단어 찾기
    for word, index in t.word_index.items():
      # print(word, index)
      if index == result: #예측한 단어의 인덱스와 동일한 단어가 있다면
        break # 해당 단어가 예측단어이므로 break

      current_word = current_word + ' ' + word
      sentence = sentence + ' ' + word
    sentence = init_word + sentence
    return sentence

# print(sequence_gen_func(model,tok,'눈이',5))
# print(sequence_gen_func(model,tok,'그의',8))
# print(sequence_gen_func(model,tok,'그의',10))
# print(sequence_gen_func(model,tok,'거리에',3))

print(sequence_gen_func(model,tok,'희망퇴직',5))
print(sequence_gen_func(model,tok,'은행권이',10))
print(sequence_gen_func(model,tok,'은행권이',20))
print(sequence_gen_func(model,tok,'은행원들은',5))


희망퇴직 희망퇴직 이상 희망퇴직을 만 수 눈치만 보는 중이다 지난달 신청을 올해 것은 퇴직금 때문이다 5대 희망 특별 퇴직금을 넘는 조건이 때 것이다 조건을 노조 상황”이라고 직원은 치 일반 은행권이 연말
은행권이 희망퇴직 이상 희망퇴직을 만 수 눈치만 보는 중이다 지난달 신청을 올해 것은 퇴직금 때문이다 5대 희망 특별 퇴직금을 넘는 조건이 때 것이다 조건을 노조 상황”이라고 직원은 치 일반 은행권이
은행권이 희망퇴직 이상 희망퇴직을 만 수 눈치만 보는 중이다 지난달 신청을 올해 것은 퇴직금 때문이다 5대 희망 특별 퇴직금을 넘는 조건이 때 것이다 조건을 노조 상황”이라고 직원은 치 일반 은행권이
은행원들은
